# 3. Regression
In this notebook we will finally run our regression models. For that purpose, we are importing the necessary libraries and functions from our ```modules``` folder. We are also importing our extracted dataframe

In [1]:
# Warnings
import warnings
warnings.filterwarnings("ignore")

# Basic Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import seaborn as sns
from scipy import stats
from functools import reduce

# Statsmodels
import statsmodels.api as sm
import pmdarima as pmd
from pmdarima.arima import auto_arima
from statsmodels.tsa.api import VAR
from statsmodels.tsa.vector_ar.var_model import VARResults
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose

# Machine Learning models
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, TimeSeriesSplit
from sklearn.linear_model import Ridge, Lasso, ElasticNet, ElasticNetCV, LinearRegression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import (
    mean_absolute_error,
    mean_squared_error,
    mean_absolute_percentage_error,
    median_absolute_error,
    r2_score,
    precision_score

)

from xgboost import XGBRegressor

In [2]:
# We import our own functions
import sys
sys.path.append('../../../')  # Move two levels up to the project root
from modules.functions import *

In [3]:
df = pd.read_csv('../../../input/df_raw_c23.csv', parse_dates=['Fecha'], index_col='Fecha')
df.tail()

,CPI Core,CPI Tradable,CPI Non-Tradable,CPI,CPI Non-Core,CPI Food and Energy,CPI Excluding Food and Energy,CPI Food and Beverages,CPI Excluding Food and Beverages,CPI Core Excluding Food and Beverages,CPI Imported,Wholesale Price Index,Reserve Requirement Rate,Monetary Policy Rate,Circulating Currency Seasonally Adjusted (mill S/),Net International Reserves (mill $),Real Minimum Wage (Index)
Fecha,,,,,,,,,,,,,,,,,
2023-08-01,0.308731,0.263821,0.147377,0.182324,-0.071459,0.175741,0.189515,0.263738,0.125240,0.215058,-0.170008,-0.104763,5.129333,7.75,-0.002583,-0.019404,-0.003747
2023-09-01,0.281829,0.237390,0.089467,0.133933,-0.163217,0.099223,0.165369,0.179682,0.102272,0.195769,-0.206148,-0.125837,5.262712,7.50,0.002269,-0.008615,-0.000164
2023-10-01,0.254555,0.210974,0.031289,0.085365,-0.254697,0.022078,0.141472,0.094521,0.079821,0.176261,-0.240150,-0.144366,5.270346,7.25,0.006678,-0.001206,0.003235
2023-11-01,0.226933,0.184472,-0.026935,0.036736,-0.345597,-0.055512,0.117886,0.008508,0.057913,0.156596,-0.272242,-0.160344,5.629236,7.00,-0.002775,0.008482,0.000000
2023-12-01,0.198984,0.157822,-0.085018,-0.011845,-0.435639,-0.133358,0.094639,-0.078087,0.036539,0.136819,-0.302719,-0.173843,5.858670,6.75,-0.013317,-0.010011,0.000000


In [4]:
df_lags = pd.read_csv('../../../input/df_lags_c23.csv', parse_dates=['Fecha'], index_col='Fecha')
df_lags.tail()

,CPI Core,CPI Tradable_lag_1,CPI Tradable_lag_2,CPI Non-Tradable_lag_1,CPI Non-Tradable_lag_2,CPI_lag_1,CPI_lag_2,CPI Non-Core_lag_1,CPI Non-Core_lag_2,CPI Food and Energy_lag_1,...,Reserve Requirement Rate_lag_1,Reserve Requirement Rate_lag_2,Monetary Policy Rate_lag_1,Monetary Policy Rate_lag_2,Circulating Currency Seasonally Adjusted (mill S/)_lag_1,Circulating Currency Seasonally Adjusted (mill S/)_lag_2,Net International Reserves (mill $)_lag_1,Net International Reserves (mill $)_lag_2,Real Minimum Wage (Index)_lag_1,Real Minimum Wage (Index)_lag_2
Fecha,,,,,,,,,,,,,,,,,,,,,
2023-08-01,0.308731,0.290249,0.316561,0.204813,0.261591,0.230389,0.277964,0.020204,0.111347,0.251427,...,5.588473,5.974649,7.75,7.75,-0.017679,-0.011038,0.004548,-0.043297,-0.003909,0.001566
2023-09-01,0.281829,0.263821,0.290249,0.147377,0.204813,0.182324,0.230389,-0.071459,0.020204,0.175741,...,5.129333,5.588473,7.75,7.75,-0.002583,-0.017679,-0.019404,0.004548,-0.003747,-0.003909
2023-10-01,0.254555,0.237390,0.263821,0.089467,0.147377,0.133933,0.182324,-0.163217,-0.071459,0.099223,...,5.262712,5.129333,7.50,7.75,0.002269,-0.002583,-0.008615,-0.019404,-0.000164,-0.003747
2023-11-01,0.226933,0.210974,0.237390,0.031289,0.089467,0.085365,0.133933,-0.254697,-0.163217,0.022078,...,5.270346,5.262712,7.25,7.50,0.006678,0.002269,-0.001206,-0.008615,0.003235,-0.000164
2023-12-01,0.198984,0.184472,0.210974,-0.026935,0.031289,0.036736,0.085365,-0.345597,-0.254697,-0.055512,...,5.629236,5.270346,7.00,7.25,-0.002775,0.006678,0.008482,-0.001206,0.000000,0.003235


## 3.1 Benchmark models

In the first section, we first run our benchmark econometric models: ```Random Walk (RW)```,  ```Autoregressive Integrated Moving Average (ARIMA)``` and ```Vector Autoregression (VAR)``` processes

### 3.1.1 Random Walk (RW)

In [5]:
forecast_horizons = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

# We define our target variable
target = 'CPI Core'

# We only use CPI as Random Walk is an univariate process
df_CPI_Core = pd.DataFrame(df_lags['CPI Core'])

# We create our train and test set
train_set = df_CPI_Core[df_CPI_Core.index < '2023-01-01']
test_set  = df_CPI_Core[df_CPI_Core.index >= '2023-01-01']

predictions = {}

for h in forecast_horizons:
    # We get the values h horizons before
    predicted_value = train_set.iloc[-h, 0]

    # We save it for horizon h
    predictions[h] = predicted_value

predicted = pd.DataFrame([predictions]).transpose().reset_index()

predicted.columns = ['Horizon', 'Prediction']

predicted = predicted.set_index(test_set.index)

predicted

,Horizon,Prediction
Fecha,,
2023-01-01,1,0.503978
2023-02-01,2,0.518182
2023-03-01,3,0.527746
2023-04-01,4,0.532433
2023-05-01,5,0.532530
2023-06-01,6,0.529124
2023-07-01,7,0.523670
2023-08-01,8,0.517117
2023-09-01,9,0.509362


In [6]:
# We create our results dataframe, concatenating the predicted and the actual values
results = pd.concat([predicted, test_set[target]], axis=1)
results.rename(columns={'Horizon': 'Horizon', 'Prediction': 'Predicted', 'CPI Core': 'Actual'}, inplace=True)
results

,Horizon,Predicted,Actual
Fecha,,,
2023-01-01,1,0.503978,0.486151
2023-02-01,2,0.518182,0.461812
2023-03-01,3,0.527746,0.436986
2023-04-01,4,0.532433,0.412057
2023-05-01,5,0.532530,0.386863
2023-06-01,6,0.529124,0.361269
2023-07-01,7,0.523670,0.335223
2023-08-01,8,0.517117,0.308731
2023-09-01,9,0.509362,0.281829


In [7]:
# We get our metrics using our function
RMSE_rw, MAPE_rw = get_metrics(results, 'RW')
metrics_rw = pd.concat([RMSE_rw, MAPE_rw], axis = 1)
metrics_rw

,RMSE_RW,MAPE_RW
1,0.017827,0.036670
2,0.041805,0.079366
3,0.062537,0.122143
4,0.080968,0.164640
5,0.097409,0.207019
6,0.112263,0.249954
7,0.125999,0.294554
8,0.138994,0.342107
9,0.151410,0.393800
10,0.163189,0.450630


### 3.1.2 Autoregressive Integrated Moving Average (ARIMA)

In [8]:
# We only use CPI as Random Walk is an univariate process
df_CPI_Core = pd.DataFrame(df_lags['CPI Core'])

# We create our train and test set
train_set = df_CPI_Core[df_CPI_Core.index < '2023-01-01']
test_set  = df_CPI_Core[df_CPI_Core.index >= '2023-01-01']

In [9]:
# We find the best SARIMA model
autoarima = pmd.auto_arima(
        y = train_set,
        start_p=1,
        start_q=0,
        seasonal=True,
        max_p=12,
        max_d=1,
        max_q=6,
        max_P=12,
        max_D=1,
        max_Q=6,
        m=4,
        n_jobs=-1,
        suppress_warnings=True,
        )

# We indicate the seasonal order for monthly data
seasonal_order = (1, 1, 1, 12)

# We create our ARIMA model
model = SARIMAX(train_set,
                order=autoarima.order,
                seasonal_order=autoarima.seasonal_order,
                enforce_stationarity = False,
                enforce_invertibility = False)
        
# We fit the model
model_fit = model.fit()

# We forecast for the next 12 horizons
forecast_values = model_fit.get_forecast(steps=12)
predicted = pd.DataFrame(forecast_values.predicted_mean, index = test_set.index)

# We create our results dataframe, concatenating the predicted and the actual values
results = pd.concat([predicted, test_set[target]], axis=1)
results.rename(columns={'predicted_mean': 'Predicted', 'CPI Core': 'Actual'}, inplace=True)
results

,Predicted,Actual
Fecha,,
2023-01-01,0.485767,0.486151
2023-02-01,0.464391,0.461812
2023-03-01,0.440750,0.436986
2023-04-01,0.415739,0.412057
2023-05-01,0.390209,0.386863
2023-06-01,0.364944,0.361269
2023-07-01,0.340646,0.335223
2023-08-01,0.317923,0.308731
2023-09-01,0.297280,0.281829


In [10]:
# We get our metrics using our function
RMSE_arima, MAPE_arima = get_metrics(results, 'ARIMA')
metrics_arima= pd.concat([RMSE_arima, MAPE_arima], axis = 1)
metrics_arima

,RMSE_ARIMA,MAPE_ARIMA
1,0.000384,0.000790
2,0.001844,0.003188
3,0.002644,0.004996
4,0.002938,0.005981
5,0.003024,0.006515
6,0.003142,0.007124
7,0.003558,0.008417
8,0.004652,0.011087
9,0.006765,0.015947
10,0.010075,0.024000


### 3.1.3 Vector autoregression (VAR)

In [11]:
# We define our target variable, as well as our train and test set
target = 'CPI Core'
train_set = df[df.index < '2023-01-01']
test_set  = df[df.index >= '2023-01-01']

In [12]:
# We model our VAR including up to two lags
model_var = VAR(df)
model_fit = model_var.fit(2)

In [13]:
# We forecast for the next 12 months
preds = model_fit.forecast(df.values[-2:], 12)
preds = pd.DataFrame(preds, index = test_set[target].index)[0]

# We create our results dataframe, concatenating the predicted and the actual values
results = pd.concat([preds, test_set[target]],axis=1)
results.rename(columns={'CPI Core': 'Actual', 0: 'Predicted'}, inplace=True)
results

,Predicted,Actual
Fecha,,
2023-01-01,0.171803,0.486151
2023-02-01,0.145264,0.461812
2023-03-01,0.119214,0.436986
2023-04-01,0.093653,0.412057
2023-05-01,0.068629,0.386863
2023-06-01,0.044232,0.361269
2023-07-01,0.020591,0.335223
2023-08-01,-0.002128,0.308731
2023-09-01,-0.023733,0.281829


In [14]:
# We get our metrics using our function
RMSE_var, MAPE_var = get_metrics(results, 'VAR')
metrics_var= pd.concat([RMSE_var, MAPE_var], axis = 1)
metrics_var

,RMSE_VAR,MAPE_VAR
1,0.314348,0.646606
2,0.315449,0.666026
3,0.316226,0.686415
4,0.316772,0.707991
5,0.317065,0.730913
6,0.317060,0.755355
7,0.316714,0.781529
8,0.315988,0.809700
9,0.314847,0.840201
10,0.313257,0.873470


## 3.2 Machine learning models

In the second section, we run our machine learning models: ```Ridge Regression (Ridge)```,  ```Least Absolute Shrinkage and Selection Operator (LASSO)``` and ```Random Forest (RF)``` models

### 3.2.1 Ridge Regression (Ridge)

In [15]:
# def test_models_regression(models, data, pred_vars, target_var ):
#     """
#     Evalúa modelos de regresión utilizando validación cruzada en series temporales.

#     Parámetros:
#     - modelos: Diccionario de modelos de regresión para evaluar.
#     - datos: DataFrame que contiene el conjunto de datos.
#     - variables_predictoras: Lista de nombres de variables predictoras.
#     - variable_objetivo: Nombre de la variable objetivo.

#     Retorna:
#     DataFrame: Resultados de la evaluación del modelo.
#     """       
#     results = {
#         'Model': [],
#         'R2_train': [],
#         'R2_test': [],
#         'MAE_train': [],
#         'MAE_test': [],
#         'MAPE_train': [],
#         'MAPE_test': [],
#         'MSE_train': [],
#         'MSE_test': [],
#         'RMSE_train': [],
#         'RMSE_test': [],
#         'Grid_Search_Params': []
#     }
    
#     X = data[pred_vars]
#     y = data[target_var]
    
#     cv = TimeSeriesSplit(n_splits=5)
    
#     print(f"Entrenando y evaluando modelos...")
    
#     for model_name, model_params in models.items():
#         print(f"Procesando el modelo: {model_name}")
        
#         if 'model' in model_params:
#             model = model_params['model']
#         else:
#             raise ValueError(f'Model is not defined for {model_name}')
        
#         if 'grid_params' in model_params:
#             grid_params = model_params['grid_params']
#         else:
#             grid_params = None
        
#         best_params = None
        
#         for ii, (tr, tt) in enumerate(cv.split(X, y)):
#             X_train, X_test = X.iloc[tr], X.iloc[tt]
#             y_train, y_test = y.iloc[tr], y.iloc[tt]
            
#             if ii == (cv.n_splits - 1):
            
#                 if grid_params is not None:
#                     grid_search = GridSearchCV(model, grid_params, cv=cv)
#                     grid_search.fit(X_train, y_train)
#                     best_model = grid_search.best_estimator_
#                     best_params = grid_search.best_params_

#                     if hasattr( best_model, 'feature_importances_' ):

#                         feature_importances = best_model.feature_importances_
#                         vars_df             = pd.DataFrame( {'Var': pred_vars, 'Importance Score': feature_importances } )
#                         vars_df             = vars_df.reindex( vars_df[ 'Importance Score' ].abs().sort_values( ascending = False ).index )
#                         vars_df.to_excel( f'varlist__{ model_name }.xlsx' )

#                     elif hasattr( best_model, 'coef_' ):

#                         coefficients = best_model.coef_[ 0 ]
#                         vars_df      = pd.DataFrame( {'Var': best_model.feature_names_in_, 'Coefficient': coefficients } )
#                         vars_df      = vars_df.reindex( vars_df[ 'Coefficient' ].abs().sort_values( ascending = False ).index )
#                         vars_df.to_excel( f'varlist_{ model_name }.xlsx' )
#                 else:
#                     best_model = model.fit(X_train, y_train)
#                     coefficients = best_model.coef_[ 0 ]
#                     vars_df      = pd.DataFrame( {'Var': best_model.feature_names_in_, 'Coefficient': coefficients } )
#                     vars_df      = vars_df.reindex( vars_df[ 'Coefficient' ].abs().sort_values( ascending = False ).index )
#                     vars_df.to_excel( f'varlist_{ model_name }.xlsx' )

#                 y_pred_train = best_model.predict(X_train)
#                 y_pred_test = best_model.predict(X_test)

#                 best_model_params = {
#                     'Model': model_name,
#                     'Grid_Search_Params': best_params
#                 }

#         results['Model'].append(best_model_params['Model'])
#         results['Grid_Search_Params'].append(best_model_params['Grid_Search_Params'])
    
#     results_df = pd.DataFrame(results)
#     results_df = results_df.sort_values(by='RMSE_test', ascending=True)

#     return results_df

In [16]:
# import pandas as pd
# from datetime import datetime, timedelta

# # Asumiendo que tu DataFrame tiene una columna 'timestamp' que representa la fecha
# # y que 'target_variable' es la variable que deseas predecir

# # 1. Filtra los datos hasta diciembre de 2018
# train_data = df[df['timestamp'] <= '2018-12-01']

# # 2. Filtra los últimos 12 meses para el conjunto de prueba
# test_start_date = datetime.strptime('2018-12-01', '%Y-%m-%d') + timedelta(days=1)
# test_data = df[(df['timestamp'] >= test_start_date) & (df['timestamp'] <= '2019-12-01')]

# # 3. Divide tus datos en características (X) y variable objetivo (y)
# X_train = train_data.drop('target_variable', axis=1)  # Ajusta 'target_variable' al nombre de tu variable objetivo
# y_train = train_data['target_variable']

# X_test = test_data.drop('target_variable', axis=1)
# y_test = test_data['target_variable']

# # Ahora tienes X_train, y_train para entrenar tu modelo hasta diciembre de 2018,
# # y X_test, y_test para evaluar tu modelo en los últimos 12 meses.
